In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import sys
sys.path.append("../")
import fast_scBatch
from tqdm import trange

In [ ]:
cell = anndata.read_h5ad(f"../../../realdata/pancreas/data.h5ad")
batch = cell.obs[["batch"]].copy()
ctype = cell.obs[["celltype"]].copy()
cells = cell.to_df().T
corr = pd.read_csv(f"../../fastqn/corr_pancreas.csv", index_col=0)
corr.columns = cells.columns
corr.index = cells.columns
p, n = cells.shape
res = fast_scBatch.solver(cells, corr, batch, p=0.3, k=50,
	lr=(0.0002, 0.0001, 0.0003), EPOCHS=(0, 0, 500), verbose=True)
adata = anndata.AnnData(X=res.T, obs=cell.obs, var=cell.var)
adata.write("../results/fastscbatch_pancreas.h5ad")

In [ ]:
print(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.pl.umap(adata, color=["batch", "celltype"], wspace=0.5)